In [76]:
import numpy as np
import xml.sax
import sys
import csv
import re
import string
import datetime
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
    

[nltk_data] Downloading package stopwords to /home/smriti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [77]:
# Titles= {}
STOPWORDS = set(stopwords.words('english')) 
URL_STOP_WORDS = set(["http", "https", "www", "ftp", "com", "net", "org", "archives", "pdf", "html", "png", "txt", "redirect"])
EXTENDED_PUNCTUATIONS = set(list(string.punctuation) + ['\n', '\t', ' '])
# INT_DIGITS = set(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"])

maxDocs_indexed_atOnce= 100
indexed_untilNow =0
current_invertedFileIndex= 1
docId=1
Wiki_Dict= {}
Doc_Dict= {}

In [78]:
def kWayMerge():
    pass

In [79]:
def create_invertedIndex():
    global current_invertedFileIndex
    sorted(Wiki_Dict.items(), key=lambda t: t[0])
    f = open("./InvertedIndices/InvertedIndex_{}.txt".format(current_invertedFileIndex), "w")
    for key, value in Wiki_Dict.items():
        line= key +':'+ value +"\n"
        f.write(line)
    f.close()
    current_invertedFileIndex +=1
    Wiki_Dict.clear()

In [80]:
# class Page:
def ProcessPage(pageID, content, index):
    for word in content:
        if word not in Doc_Dict:
            value= np.zeros(6, dtype= int)
            Doc_Dict[word]= value
            Doc_Dict[word][index] =1
        else:
            Doc_Dict[word][index] +=1
    
    if(index ==5):
#         print (Doc_Dict)
        for key, value in Doc_Dict.items():
            if key not in Wiki_Dict:                # pageID_totalcount:t1i2c3b4r5l6|pageID_totalcount:t1i2c3b4r5l6
                Wiki_Dict[key]= str(pageID)+ '-'+ 't'+ str(value[0])+ 'i'+ str(value[1])+ 'c'+ str(value[2])+ 'b'+ str(value[3])+ 'r'+ str(value[4])+ 'l'+ str(value[5])
            else:
                Wiki_Dict[key] += '|'+ str(pageID)+ '-'+ 't'+ str(value[0])+ 'i'+ str(value[1])+ 'c'+ str(value[2])+ 'b'+ str(value[3])+ 'r'+ str(value[4])+ 'l'+ str(value[5])
        
        global maxDocs_indexed_atOnce, indexed_untilNow
    
        if(maxDocs_indexed_atOnce == indexed_untilNow):
            create_invertedIndex()
            indexed_untilNow =0 
        indexed_untilNow +=1
        Doc_Dict.clear()
        
    
    
    
def extract_pageDetails(text):
    lines= text.split('\n')
    line =0
    infoBox, category, body, references, links= [], [], [], [], []

    while line <len(lines):
        if "{{infobox" in lines[line]:
            openBrackets, closedBrackets= 0,0
            while line< len(lines):
                if "{{" in lines[line]:
                    openBrackets +=1
                if "}}" in lines[line]:
                    closedBrackets +=1

                infoBox.extend(lines[line].split())
                if(openBrackets == closedBrackets):
                    break
                line +=1
        elif "[[category:" in lines[line]:
            try:
                category.extend((lines[line].split(':')[1].split(']]')[0]).split())
            except:
                category.extend((lines[line].split(':')[1]).split())

        elif "==external links==" in lines[line] or "== external links ==" in lines[line]:
            line +=1
            while line< len(lines):
                if "*[" in lines[line] or "* [" in lines[line]:
                    link= ""
                    while line< len(lines) and "]" not in lines[line]:
                        link += lines[line]
                        line+=1
                    link += lines[line]
                    link = link.split('[')
                    if(len(link)> 1):
                        link= link[1].split(']')[0]
                        links.extend(link.split())
                elif "[[category:" in lines[line]:
                    try:
                        category.extend((lines[line].split(':')[1].split(']]')[0]).split())
                    except:
                        category.extend((lines[line].split(':')[1]).split())
                else:
                    body.extend(lines[line].split())
                line+=1
                

        elif "==references==" in lines[line] or "== references ==" in lines[line]:
            line +=1
            while line< len(lines):
                if "==" in lines[line] or "[[category:" in lines[line]:
                    line -=1
                    break
                elif "{{cite" in lines[line] or "{{vcite" in lines[line]:
                    cite_title= lines[line].split("title=")
                    if(len(cite_title) >1):
                        cite_title= cite_title[1].split("|")[0]
                        references.extend(cite_title.split())
                elif "{{" in lines[line] and "ref" not in lines[line]:
                    references.extend((lines[line].split("{{")[1].split("}}")[0]).split())
                line+=1
        else:
            body.extend(lines[line].split())
        line +=1  
    
    print (infoBox, category, body, references, links)
    return ' '.join(infoBox), ' '.join(category), ' '.join(body), ' '.join(references), ' '.join(links)

In [81]:
class Data:
    def __init__(self):
        self.docId = 0
        self.title=""
        self.text = ""
        self.infoBox= ""
        self.category= ""
        self.body= ""
        self.references= ""
        self.links= ""
        self.stemmer= SnowballStemmer("english")

    def set_data(self, docId, title, text):
        self.docId = docId
        self.title= title
        self.text = text

        self.infoBox, self.category, self.body, self.references, self.links = extract_pageDetails(self.text)
        
        self.title= self.cleanText(self.title)
        self.infoBox= self.cleanText(self.infoBox)
        self.category= self.cleanText(self.category)
        self.body= self.cleanText(self.body)
        self.references= self.cleanText(self.references)
        self.links= self.cleanText(self.links)
        
#         self.remove_numbers()
#         self.cleanText()
#         self.remove_punctuations()
        self.strip_tokenize()
        self.remove_stopWords()
        self.stemming()
        
        ProcessPage(self.docId, self.title, 0)
        ProcessPage(self.docId, self.infoBox, 1)
        ProcessPage(self.docId, self.category, 2)
        ProcessPage(self.docId, self.body, 3)
        ProcessPage(self.docId, self.references, 4)
        ProcessPage(self.docId, self.links, 5)


    def remove_numbers(self):
        self.title = re.sub(r'\\d+', '', self.title)
        self.infoBox = re.sub(r'\\d+', '', self.infoBox)
        self.category = re.sub(r'\\d+', '', self.category)
        self.body = re.sub(r'\\d+', '', self.body)
        self.references = re.sub(r'\\d+', '', self.references)
        self.links = re.sub(r'\\d+', '', self.links)
    
    def cleanText(self, text):
        text = re.sub(r'<(.*?)>','',text) #Remove tags if any
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE) #Remove Url
        text = re.sub(r'{\|(.*?)\|}', '', text, flags=re.MULTILINE) #Remove CSS
        text = re.sub(r'\[\[file:(.*?)\]\]', '', text, flags=re.MULTILINE) #Remove File
        text = re.sub(r'[.,;_()"/\'=]', ' ', text, flags=re.MULTILINE) #Remove Punctuaion
        text = re.sub(r'[~`!@#$%&-^*+{\[}\]()":\|\\<>/?]', ' ', text, flags=re.MULTILINE)
        return ' '.join(text.split())

    def remove_punctuations(self):
        self.title = self.title.translate(str.maketrans('', '', string.punctuation))
        self.infoBox = self.infoBox.translate(str.maketrans('', '', string.punctuation))
        self.category = self.category.translate(str.maketrans('', '', string.punctuation))
        self.body = self.body.translate(str.maketrans('', '', string.punctuation))
        self.references = self.references.translate(str.maketrans('', '', string.punctuation))
        self.links = self.links.translate(str.maketrans('', '', string.punctuation))

    def strip_tokenize(self):
        self.title= self.title.strip().split()
        self.infoBox= self.infoBox.strip().split()
        self.category= self.category.strip().split()
        self.body= self.body.strip().split()
        self.references= self.references.strip().split()
        self.links= self.links.strip().split()

    def remove_stopWords(self):
        self.title = [x for x in self.title if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.infoBox = [x for x in self.infoBox if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.category = [x for x in self.category if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.body = [x for x in self.body if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.references = [x for x in self.references if x not in STOPWORDS and x not in URL_STOP_WORDS]
        self.links = [x for x in self.links if x not in STOPWORDS and x not in URL_STOP_WORDS]


    def stemming(self):
        self.title = [self.stemmer.stem(line) for line in self.title]
        self.infoBox = [self.stemmer.stem(line) for line in self.infoBox]
        self.category = [self.stemmer.stem(line) for line in self.category]
        self.body = [self.stemmer.stem(line) for line in self.body]
        self.references = [self.stemmer.stem(line) for line in self.references]
        self.links = [self.stemmer.stem(line) for line in self.links]


In [82]:
class WikiHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.CurrentData = ""
        self.title = ""
        self.text = ""

    # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        global docId
        if tag == "page":
            print (docId, end= " ")

    # Call when an elements ends
    def endElement(self, tag):
        global docId
        if self.CurrentData == "title":
#             Titles[docId]= self.title
            print ("title: ", self.title)
        elif self.CurrentData == "text":
            page= Data()
            page.set_data(docId, self.title, self.text)
            docId +=1
        self.text= ""
        self.CurrentData = ""
#         if(docId ==50):
#             sorted(Wiki_Dict.items(), key=lambda t: t[0])
#             print (Wiki_Dict)
#             sys.exit()


    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "title":
            self.title = content.lower()
        elif self.CurrentData == "text":
            self.text += content.lower()


In [83]:
if ( __name__ == "__main__"):
    # create an XMLReader
    parser = xml.sax.make_parser()
    # turn off namepsaces
    parser.setFeature(xml.sax.handler.feature_namespaces, 0)

    # override the default ContextHandler
    Handler = WikiHandler()
    parser.setContentHandler(Handler)

    start = datetime.datetime.now()

    dump_data= "../Phase1/wikipedia.xml"
    parser.parse(dump_data)

    end = datetime.datetime.now()
    secs  = (end-start).seconds
    hr = int(secs/(60*60))
    rm = int(secs%(60*60))
    mn = int(rm/60)
    rm=int(rm%60)
    secs = int(rm)
    print("Indexing Time : ",hr," hrs ",mn," mns",secs," secs")


1 title:  accessiblecomputing
[] [] ['#redirect', '[[computer', 'accessibility]]', '{{r', 'from', 'move}}', '{{r', 'from', 'camelcase}}', '{{r', 'unprintworthy}}'] [] []
2 title:  anarchism
[] ['anarchism|', 'anti-capitalism', 'anti-fascism', 'economic', 'ideologies', 'far-left', 'politics', 'libertarian', 'socialism', 'libertarianism', 'political', 'culture', 'political', 'movements', 'political', 'ideologies', 'social', 'theories'] ['{{short', 'description|political', 'philosophy', 'and', 'movement}}', '{{redirect2|anarchist|anarchists|other', 'uses|anarchists', '(disambiguation)}}', '{{pp-move-indef}}', '{{good', 'article}}', '{{use', 'dmy', 'dates|date=march', '2020}}', '{{use', 'british', 'english|date=january', '2014}}', '{{anarchism', 'sidebar}}', '{{basic', 'forms', 'of', 'government}}', "'''anarchism'''", 'is', 'a', '[[political', 'philosophy]]', 'and', '[[political', 'movement|movement]]', 'that', 'rejects', 'all', 'involuntary,', 'coercive', 'forms', 'of', '[[hierarchy]].', 

13 title:  albaniahistory
[] [] ['#redirect', '[[history', 'of', 'albania]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
14 title:  albaniapeople
[] [] ['#redirect', '[[demographics', 'of', 'albania]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
15 title:  aswemaythink
[] [] ['#redirect', '[[as_we_may_think]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
16 title:  albaniagovernment
[] [] ['#redirect', '[[politics', 'of', 'albania]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
17 title:  albaniaeconomy
[] [] ['#redirect', '[[economy', 'of', 'albania]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
18 title:  albedo
[] ['land', 'surface', 'effects', 'on', 'climate', 'climate', 'change', 'feedbacks', 'climate', 'forcing', 'climatology', 'electromagnetic', 'radiation', 'radiometry', 'scattering,', 'absorption'

['{{infobox', 'u.s.', 'state', '|official_name', '=', 'state', 'of', 'alabama', '|name', '=', 'alabama', '|image_flag', '=', 'flag', 'of', 'alabama.svg', '|flag_link', '=', 'flag', 'of', 'alabama', '|image_seal', '=', 'seal', 'of', 'alabama.svg', '|seallink', '=', 'seal', 'of', 'alabama', '|nickname', '=', 'the', '[[northern', 'flicker|yellowhammer]]', 'state,', 'the', 'heart', 'of', '[[dixie]],', 'and', 'the', 'cotton', 'state', '|motto', '=', '{{lang-la|[[audemus', 'jura', 'nostra', 'defendere]]}}', '<br>(we', 'dare', 'defend', 'our', 'rights)', '|anthem', '=', '"[[alabama', '(state', 'song)|alabama]]"', '|image_map', '=', 'alabama', 'in', 'united', 'states.svg', '|seat', '=', '[[montgomery,', 'alabama|montgomery]]', '|largestcity', '=', '[[birmingham,', 'alabama|birmingham]]', '|largestcounty', '=', '[[baldwin', 'county,', 'alabama|baldwin', 'county]]', '|largestmetro', '=', '[[birmingham', 'metropolitan', 'area,', 'alabama|greater', 'birmingham]]', '|area_total_km2', '=', '135,765'

67 title:  africa
[] [] ['#redirect', '[[africa]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'camelcase}}', '}}'] [] []
68 title:  achilles
[] ['achilles|', 'demigods', 'of', 'classical', 'mythology', 'achaean', 'leaders', 'kings', 'of', 'the', 'myrmidons', 'thessalians', 'in', 'the', 'trojan', 'war', 'greek', 'mythological', 'heroes', 'lgbt', 'themes', 'in', 'greek', 'mythology', 'fictional', 'lgbt', 'characters', 'in', 'literature'] ['{{redirect|achilleus|the', 'roman', 'emperor', 'with', 'this', 'name|achilleus', '(roman', 'usurper)|other', 'uses|achilles', '(disambiguation)}}', '{{short', 'description|greek', 'mythological', 'hero}}', '{{use', 'dmy', 'dates|date=april', '2020}}', '[[file:achilles', 'fighting', 'against', 'memnon', 'leiden', 'rijksmuseum', 'voor', 'oudheden.jpg|thumb|260px|ancient', 'greek', 'polychromatic', '[[ancient', 'greek', 'vase', 'painting|pottery', 'painting]]', '(dating', 'to', '{{circa}}', '300', 'bc)', 'of', 'achilles', 'during', 'the', 'troj

71 title:  aristotle
['{{infobox', 'philosopher', '|', 'name=aristotle', '|', 'image=aristotle', 'altemps', 'inv8575.jpg', '|', 'caption=roman', 'copy', 'in', 'marble', 'of', 'a', 'greek', 'bronze', '[[bust', '(sculpture)|bust]]', 'of', 'aristotle', 'by', '[[lysippos]],', '[[circa|c.]]', '330', 'bc,', 'with', 'modern', '[[alabaster]]', '[[mantle', '(clothing)|mantle]]', '|', 'birth_date=384', 'bc{{efn-ua|that', 'these', 'dates', '(the', 'first', 'half', 'of', 'the', 'olympiad', 'year', '384/383', 'bc,', 'and', 'in', '322', 'shortly', 'before', 'the', 'death', 'of', 'demosthenes)', 'are', 'correct', 'was', 'shown', 'by', '[[august', 'boeckh]]', "(''kleine", "schriften''", 'vi', '195);', 'for', 'further', 'discussion,', 'see', '[[felix', 'jacoby]]', 'on', "''[[fgrhist]]''", '244', 'f', '38.', 'ingemar', 'düring,', "''aristotle", 'in', 'the', 'ancient', 'biographical', "tradition'',", 'göteborg,', '1957,{{p.|253}}}}', '|', 'birth_place=[[stagira', '(ancient', 'city)|stagira]],', '[[chalci

...

75 title:  action film
[] [] ['#redirect', '[[action', 'film]]', '{{redirect', 'category', 'shell|1=', '{{r', 'from', 'other', 'capitalisation}}', '}}'] [] []
76 title:  actrius
['{{infobox', 'film', '|', 'name', '=', 'actresses', '|', 'image', '=', 'actrius', 'film', 'poster.jpg', '|', 'alt', '=', '|', 'caption', '=', 'catalan', 'language', 'film', 'poster', '|', 'native_name', '=', '([[catalan', 'language|catalan]]:', "'''''actrius''''')", '|', 'director', '=', '[[ventura', 'pons]]', '|', 'producer', '=', 'ventura', 'pons', '|', 'writer', '=', '[[josep', 'maria', 'benet', 'i', 'jornet]]', '|', 'screenplay', '=', 'ventura', 'pons', '|', 'story', '=', '|', 'based_on', '=', '{{based', 'on|(stage', 'play)', "''e.r.''|josep", 'maria', 'benet', 'i', 'jornet}}', '|', 'starring', '=', '{{ubl|[[núria', 'espert]]|[[rosa', 'maria', 'sardà]]|[[anna', 'lizaran]]|[[mercè', 'pons]]}}', '|', 'narrator', '=', '<!--', 'or:', '|narrators', '=', '-->', '|', 'music', '=', 'carles', 'cases', '|', 'cinemat

['{{infobox', 'writer', '|', 'name', '=', 'ayn', 'rand', '|', 'native_name', '=', 'алиса', 'зиновьевна', 'розенбаум<br>{{nowrap|(alisa', 'zinovyevna', 'rosenbaum)}}', '|', 'image', '=', 'ayn', 'rand', '(1943', 'talbot', 'portrait).jpg', '|', 'alt', '=', 'black', 'and', 'white', 'photo', 'of', 'a', 'white', 'woman', 'facing', 'the', 'camera', 'with', 'her', 'body', 'turned', 'to', 'the', 'side.', '|', 'caption', '=', 'rand', 'in', '1943', '|', 'birth_date', '=', '{{birth', 'date|mf=yes|1905|02|02}}', '|', 'birth_place', '=', '[[saint', 'petersburg|st.', 'petersburg]],', '[[russian', 'empire]]', '|', 'death_date', '=', '{{death', 'date', 'and', 'age|mf=yes|1982|3|6|1905|2|2}}', '|', 'death_place', '=', '[[manhattan]],', '[[new', 'york', 'city]],', '[[united', 'states|u.s.]]', '|', 'resting_place', '=', '[[kensico', 'cemetery]],', '[[valhalla,', 'new', 'york]],', 'united', 'states', '|', 'pseudonym', '=', 'ayn', 'rand', '|', 'occupation', '=', 'writer', '|', 'language', '=', 'english', '|

87 title:  list of atlas shrugged characters
[] ['atlas', 'shrugged', 'characters|', 'fictional', 'socialites', 'lists', 'of', 'literary', 'characters', 'literary', 'characters', 'introduced', 'in', '1957'] ['{{displaytitle:list', 'of', "''atlas", "shrugged''", 'characters}}{{more', 'footnotes|date=march', '2019}}', 'this', 'is', 'a', 'list', 'of', 'characters', 'in', '[[ayn', "rand]]'s", '1957', 'novel', "''[[atlas", "shrugged]].''", '==major', 'characters==', 'the', 'following', 'are', 'major', 'characters', 'from', 'the', 'novel.{{refn|group=note|characters', 'in', 'this', 'article', 'are', 'listed', 'as', '"major"', 'if', 'they', 'meet', 'one', 'of', 'the', 'following', 'criteria:', '*they', 'are', 'listed', 'as', '"major"', 'characters', 'in', 'a', 'widely', 'available', 'study', 'guide,', 'such', 'as', '[[cliffsnotes]],', '[[sparknotes]],', 'or', '[[gale', "(cengage)|gale's]]", "''novels", 'for', "students'';", '*they', 'are', 'listed', 'as', '"primary', 'heroic"', 'or', '"arch-v

90 title:  agricultural science
[] ['agronomy', 'agricultural', 'research|science'] ['{{short', 'description|academic', 'field', 'within', 'biology}}', '{{more', 'citations', 'needed|date=march', '2018}}', '{{use', 'dmy', 'dates|date=june', '2013}}', '{{agriculture|related}}', "'''agricultural", "science'''", 'is', 'a', 'broad', 'multidisciplinary', 'field', 'of', '[[biology]]', 'that', 'encompasses', 'the', 'parts', 'of', 'exact,', 'natural,', 'economic', 'and', '[[social', 'science]]s', 'that', 'are', 'used', 'in', 'the', 'practice', 'and', 'understanding', 'of', '[[agriculture]].', '([[veterinary', 'medicine|veterinary', 'science]],', 'but', 'not', '[[animal', 'science]],', 'is', 'often', 'excluded', 'from', 'the', 'definition.)', '==agriculture,', 'agricultural', 'science,', 'and', 'agronomy==', 'the', 'three', 'terms', 'are', 'often', 'confused.', 'however,', 'they', 'cover', 'different', 'concepts:', "*'''agriculture'''", 'is', 'the', 'set', 'of', 'activities', 'that', 'transform

KeyboardInterrupt: 